### 1. Framework imports

In [1]:
import os
import time
import pickle
import tqdm
import pandas as pd
import logging

import torch
import numpy as np

import threading
import pynvml

from pykeen.pipeline import pipeline
from pykeen.datasets import get_dataset, dataset_resolver

logging.getLogger("pykeen").setLevel(logging.ERROR)


### 2. Defining function for training and testing

In [2]:
def gpu_monitor(stop_event, interval=1.0, device_index=0, stats=None):
    """Thread para monitorar consumo de GPU durante o treino."""
    
    handle = pynvml.nvmlDeviceGetHandleByIndex(device_index)
    while not stop_event.is_set():
        mem = pynvml.nvmlDeviceGetMemoryInfo(handle)
        util = pynvml.nvmlDeviceGetUtilizationRates(handle)
        power = pynvml.nvmlDeviceGetPowerUsage(handle) / 1000.0  # W
        stats["mem_used"].append(mem.used / 1024**2)  # MB
        stats["util"].append(util.gpu)  # %
        stats["power"].append(power)  # W
        time.sleep(interval)

def run_experiment(model_name: str, dataset_name: str, epochs: int = 100,
                   batch_size: int = 256, device: str = "cuda",
                   inference_batch_size: int = 1024, seed: int = 42,
                   n_tests: int = 1, verbose: bool = True,
                   gpu_index: int = 0, monitor_interval: float = 0.1) -> pd.DataFrame:

    metrics = []
    models = []
    pynvml.nvmlInit()
    for i in range(n_tests):
        stats = {"mem_used": [], "util": [], "power": []}
        stop_event = threading.Event()
        monitor_thread = threading.Thread(
            target=gpu_monitor, args=(stop_event, monitor_interval, gpu_index, stats)
        )
        seed = seed + i
        np.random.seed(seed)

        # --- iniciar monitoramento GPU
        monitor_thread.start()

        # --- Treino + avaliação do PyKEEN
        result = pipeline(
            model=model_name,
            dataset=dataset_name,
            epochs=epochs,
            device=device,
            random_seed=seed,
            training_kwargs=dict(batch_size=batch_size, sampler="schlichtkrull", use_tqdm_batch=False),use_tqdm=verbose
        )

        # --- parar monitoramento
        stop_event.set()
        monitor_thread.join()

        # --- estatísticas GPU
        avg_mem = sum(stats["mem_used"]) / len(stats["mem_used"]) if stats["mem_used"] else None
        peak_mem = max(stats["mem_used"]) if stats["mem_used"] else None
        avg_util = sum(stats["util"]) / len(stats["util"]) if stats["util"] else None
        peak_util = max(stats["util"]) if stats["util"] else None
        avg_power = sum(stats["power"]) / len(stats["power"]) if stats["power"] else None
        peak_power = max(stats["power"]) if stats["power"] else None
        total_energy_wh = sum(p * monitor_interval for p in stats["power"]) / 3600 if stats["power"] else None

        # extrair tempos do pipeline
        train_time = getattr(result, "train_seconds", None)
        eval_time = getattr(result, "evaluate_seconds", None)

        # métricas do teste
        mrr = result.metric_results.get_metric('both.realistic.inverse_harmonic_mean_rank')
        hits1 = result.metric_results.get_metric('both.realistic.hits_at_1')
        hits3 = result.metric_results.get_metric('both.realistic.hits_at_3')
        hits5 = result.metric_results.get_metric('both.realistic.hits_at_5')
        hits10 = result.metric_results.get_metric('both.realistic.hits_at_10')

         # --- Tempo de inferência pura
        dataset = dataset_resolver.lookup(dataset_name)()
        triples = dataset.testing.mapped_triples
        n_test = int(triples.shape[0]) if hasattr(triples, "shape") else len(triples)

        model = result.model
        device_torch = model.device
        # CONVERSÃO ROBUSTA:
        if isinstance(triples, torch.Tensor):
            triples_tensor = triples.to(device=device_torch, dtype=torch.long)
        else:
            # as_tensor evita cópia se já for tensor; em numpy -> cria tensor
            triples_tensor = torch.as_tensor(triples, dtype=torch.long)
            triples_tensor = triples_tensor.to(device=device_torch)

        # inferência
        with torch.inference_mode():
            infer_t0 = time.perf_counter()
            for j in range(0, n_test, inference_batch_size):   # use 'j' para não conflitar
                batch = triples_tensor[j:j+inference_batch_size]
                _ = model.score_hrt(batch)
            # sincroniza só se CUDA
            if device_torch.type == "cuda" and torch.cuda.is_available():
                torch.cuda.synchronize(device_torch)
            infer_t1 = time.perf_counter()

        infer_time = (infer_t1 - infer_t0) / n_test if n_test else None

        # --- montar resultado em forma de DataFrame (1 linha)
        df = pd.DataFrame([{
            "model": model_name,
            "dataset": dataset_name,
            "seed": seed,
            "epochs": epochs,
            "train_time": train_time,
            "eval_time": eval_time,
            "inference_time": infer_time,
            "mrr": mrr,
            "hits@1": hits1,
            "hits@3": hits3,
            "hits@5": hits5,
            "hits@10": hits10,
            "gpu_mem_avg_MB": avg_mem,
            "gpu_mem_peak_MB": peak_mem,
            "gpu_util_avg_%": avg_util,
            "gpu_util_peak_%": peak_util,
            "gpu_power_avg_W": avg_power,
            "gpu_power_peak_W": peak_power,
            "gpu_energy_Wh": total_energy_wh,
        }])
        metrics.append(df)
        models.append(result)
    pynvml.nvmlShutdown()
        
        
    return pd.concat(metrics, ignore_index=True), models

In [6]:
# Exemplo de uso
df, models = run_experiment(model_name="ConvKB", dataset_name="DBpedia50", epochs=100, device="cuda", seed=1, n_tests = 1, inference_batch_size=1)
df

Training epochs on cuda:0:   0%|          | 0/100 [00:00<?, ?epoch/s]

KeyboardInterrupt: 

In [6]:
# Exemplo de uso
df, models = run_experiment(model_name="ConvE", dataset_name="DBpedia50", epochs=100, device="cuda", seed=1, n_tests = 5, inference_batch_size=1)
df

Training epochs on cuda:0:   0%|          | 0/100 [00:00<?, ?epoch/s]

KeyboardInterrupt: 

### 3. Definir modelos e datasets a serem avaliados

In [ ]:
model_names = ["ConvE","ConvKB","DistMult","TransE","RotatE","R-GCN"]

dataset_names = [ "DBpedia50"]
final_df = None
models = []
output_dir = 'results/'
version = 'v03'
for dataset in dataset_names:
    for model_name in model_names:
        print(f"Running {model_name} on {dataset}")
        df, model = run_experiment(model_name=model_name, dataset_name=dataset,batch_size=1024, epochs=1, device="cuda", seed=1, n_tests = 5, inference_batch_size=1)
        final_df = pd.concat([final_df, df], ignore_index=True)
        with open(output_dir + 'model_'+model_name+'_'+dataset+'.pkl', 'wb') as f:
            pickle.dump(model, f)
        del model
        final_df.to_csv(output_dir + 'results_' + version + '.csv', index=False)
        
        

Running ConvE on DBpedia50


Training epochs on cuda:0:   0%|          | 0/1 [00:00<?, ?epoch/s]

Evaluating on cuda:0:   0%|          | 0.00/2.10k [00:00<?, ?triple/s]

In [56]:
final_df

,model,dataset,seed,epochs,train_time,eval_time,inference_time,mrr,hits@1,hits@3,hits@5,hits@10,gpu_mem_avg_MB,gpu_mem_peak_MB,gpu_util_avg_%,gpu_util_peak_%,gpu_power_avg_W,gpu_power_peak_W,gpu_energy_Wh
0,ComplEx,Nations,1,100,55.889957,0.049483,0.000453,0.388256,0.171642,0.462687,0.681592,0.942786,1838.424915,2151.832031,17.493976,37,10.195176,33.216,0.117528
1,ComplEx,Nations,2,100,57.121391,0.060622,0.000376,0.392444,0.179104,0.465174,0.691542,0.947761,1834.842898,1845.843750,28.345455,39,9.852670,10.285,0.120422
2,ComplEx,Nations,4,100,56.744441,0.049614,0.000455,0.426237,0.226368,0.504975,0.691542,0.945274,1832.942903,1842.906250,26.528037,38,9.479516,9.995,0.112701
3,ComplEx,Nations,7,100,56.815753,0.055427,0.000489,0.380729,0.161692,0.460199,0.701493,0.967662,1833.139165,1841.406250,28.376168,39,9.530516,9.903,0.113307
4,ComplEx,Nations,11,100,57.151320,0.048009,0.000407,0.403539,0.196517,0.472637,0.699005,0.962687,1831.398524,1841.593750,27.121413,40,9.590554,10.011,0.120681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,ComplEx,DBpedia50,16,100,3110.717619,49.950012,0.000423,0.002023,0.000239,0.001909,0.003103,0.004535,2422.467026,11964.398438,25.473166,100,18.702320,57.506,16.242965
76,ComplEx,DBpedia50,22,100,3098.839153,38.222368,0.000411,0.001632,0.000239,0.001193,0.001671,0.003103,2441.924034,11964.398438,24.595513,100,19.163355,57.917,16.513489
77,ComplEx,DBpedia50,29,100,3102.416068,45.522542,0.000430,0.002852,0.000955,0.002625,0.003819,0.005012,2470.003624,11958.617188,24.102796,100,18.975552,57.016,16.403310
78,ComplEx,DBpedia50,37,100,3107.673787,45.633842,0.000401,0.001730,0.000239,0.001193,0.001909,0.003341,2518.112597,11958.617188,24.203248,100,18.874005,56.580,16.335975


In [58]:
final_df.to_csv('results_nations+db.csv', index=False)

In [19]:
with open('models.pkl', 'wb') as file:
    # Use pickle.dump() to serialize the model and save it to the file
    pickle.dump(models, file)